In [190]:
import glob
# This file should run basically insta fast. Bad coding for sure 
# There is a place where i sometimes comment out lin/rbf/sigmoid stuff
# Dont do that, it really helps for some subjects. 
# Anything. Leave only linear. For fast testing of best features
# But dont do that either. 

genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "08" # Which month and days result you want to visualize in table
month = 11

repetitionValue = 52
repetitionName =   "udrliplotnoAda4"   # "udrliplotnoAda3"  #  "udrlBC4CVTest" # 
#  "udrliplotnoAda3hyperparams" 
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.1
print(len(savedResults))
print(len(savedResults)/9)

90
10.0


In [191]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                
                
                # if "lin" in akernel:
                #     continue
                # if "sig" in akernel:
                #     continue
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
nameArray = np.array(list(nameDict))
topPerFeature = np.zeros([10, len(nameDict)])
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]
            
            comboName = list(nameDict.keys())[combinationList[res[0]][0]]
            nameIndex = np.where(nameArray == comboName)[0][0]
            if res[1] > topPerFeature[subNr, nameIndex]:
                topPerFeature[subNr, nameIndex] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 669)
(10, 669)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,N

In [192]:
x = 0

pd.set_option("display.max_columns", None)
print()
nameDictNrs = np.ones([1, len(nameDict)])
nameDictNrs[0,:] = np.arange(len(nameDict)) * 3
print(nameDictNrs.shape)
#table = pd.DataFrame(nameDictNrs , columns=nameDict.keys())
table = pd.DataFrame(topPerFeature, columns=nameDict.keys())
#display(table)
display(table)
# for names in nameDict.keys():
#     print(names, end="        ")
#     x +=1
#     #print(x)



(1, 223)


,dataHRCV-BCcn3-welchDataBC--welchData,welchDatacn3BC-fftDatacn3BC--welchData,gaussianDatacn3BC-fftDataBC--dataCorr1d,fftDatacn3BC-dataHRcn3--dataCorr1d,fftDatacn3BC-dataHRCV-BCcn3--dataCorr1d,dataCorr1dcn3BC-fftDatacn3--fftDataBC,welchDatacn3-dataGCVBC--welchDataBC,welchDatacn3BC-dataHRcn3--welchDataBC,dataHRCV-BCcn3-dataWCV-BCcn3--welchDataBC,dataHRCV-BCcn3-welchDataBC-welchData--fftData,welchDatacn3BC-fftDatacn3BC-welchData--fftData,gaussianDatacn3BC-fftDataBC-dataCorr1d--fftData,fftDatacn3BC-dataHRcn3-dataCorr1d--fftData,fftDatacn3BC-dataHRCV-BCcn3-dataCorr1d--fftData,dataCorr1dcn3BC-fftDatacn3-fftDataBC--fftData,welchDatacn3-dataGCVBC-welchDataBC--fftData,welchDatacn3BC-dataHRcn3-welchDataBC--fftData,dataHRCV-BCcn3-dataWCV-BCcn3-welchDataBC--fftData,dataHRCV-BCcn3-welchDataBC-dataHR--welchData,welchDatacn3BC-fftDatacn3BC-dataHR--welchData,gaussianDatacn3BC-fftDataBC-dataCorr1d--welchData,dataHRCV-BCcn3-welchDataBC-dataCorr1d--welchData,fftDatacn3BC-dataHRcn3-dataCorr1d--welchData,fftDatacn3BC-dataHRCV-BCcn3-dataCorr1d--welchData,welchDatacn3BC-fftDatacn3BC-dataCorr1d--welchData,dataHRCV-BCcn3-welchDataBC-dataFFTCV-BC--welchData,welchDatacn3BC-fftDatacn3BC-dataFFTCV-BC--welchData,dataHRCV-BCcn3-welchDataBC-dataWCV-BC--welchData,welchDatacn3BC-fftDatacn3BC-dataWCV-BC--welchData,dataHRCV-BCcn3-welchDataBC-dataHRCV-BC--welchData,welchDatacn3BC-fftDatacn3BC-dataHRCV-BC--welchData,dataHRCV-BCcn3-welchDataBC-fftDataBC--welchData,dataCorr1dcn3BC-fftDatacn3-fftDataBC--welchData,welchDatacn3BC-fftDatacn3BC-fftDataBC--welchData,dataHRCV-BCcn3-dataHRBC-welchDataBC--welchData,dataHRCV-BCcn3-gaussianData-welchDataBC--welchData,welchDatacn3-dataGCVBC-welchDataBC--welchData,dataHRCV-BCcn3-dataGCVBC-welchDataBC--welchData,dataHRCV-BCcn3-gaussianDataBC-welchDataBC--welchData,dataHRCV-BCcn3-dataCorr1dBC-welchDataBC--welchData,dataHRCV-BCcn3-fftDatacn3-welchDataBC--welchData,dataHRCV-BCcn3-welchDatacn3-welchDataBC--welchData,dataHRCV-BCcn3-dataHRcn3-welchDataBC--welchData,welchDatacn3BC-dataHRcn3-welchDataBC--welchData,dataHRCV-BCcn3-dataCorr1dcn3-welchDataBC--welchData,dataHRCV-BCcn3-dataFFTCV-BCcn3-welchDataBC--welchData,dataHRCV-BCcn3-dataWCV-BCcn3-welchDataBC--welchData,fftDatacn3BC-dataHRCV-BCcn3-welchDataBC--welchData,welchDatacn3BC-dataHRCV-BCcn3-welchDataBC--welchData,dataHRcn3BC-dataHRCV-BCcn3-welchDataBC--welchData,gaussianDatacn3-dataHRCV-BCcn3-welchDataBC--welchData,gaussianDatacn3BC-dataHRCV-BCcn3-welchDataBC--welchData,dataCorr1dcn3BC-dataHRCV-BCcn3-welchDataBC--welchData,welchDatacn3BC-fftDatacn3BC-welchDataBC--welchData,welchDatacn3BC-fftDatacn3BC-dataHRBC--welchData,welchDatacn3BC-fftDatacn3BC-gaussianData--welchData,welchDatacn3BC-fftDatacn3BC-dataGCVBC--welchData,welchDatacn3BC-fftDatacn3BC-gaussianDataBC--welchData,welchDatacn3BC-fftDatacn3BC-dataCorr1dBC--welchData,welchDatacn3BC-fftDatacn3BC-fftDatacn3--welchData,welchDatacn3BC-fftDatacn3BC-welchDatacn3--welchData,welchDatacn3BC-fftDatacn3BC-dataHRcn3--welchData,welchDatacn3BC-fftDatacn3BC-dataCorr1dcn3--welchData,welchDatacn3BC-fftDatacn3BC-dataFFTCV-BCcn3--welchData,welchDatacn3BC-fftDatacn3BC-dataWCV-BCcn3--welchData,welchDatacn3BC-fftDatacn3BC-dataHRCV-BCcn3--welchData,dataHRcn3BC-welchDatacn3BC-fftDatacn3BC--welchData,gaussianDatacn3-welchDatacn3BC-fftDatacn3BC--welchData,gaussianDatacn3BC-welchDatacn3BC-fftDatacn3BC--welchData,dataCorr1dcn3BC-welchDatacn3BC-fftDatacn3BC--welchData,gaussianDatacn3BC-fftDataBC-dataCorr1d--dataHR,fftDatacn3BC-dataHRcn3-dataCorr1d--dataHR,fftDatacn3BC-dataHRCV-BCcn3-dataCorr1d--dataHR,dataCorr1dcn3BC-fftDatacn3-fftDataBC--dataHR,welchDatacn3-dataGCVBC-welchDataBC--dataHR,welchDatacn3BC-dataHRcn3-welchDataBC--dataHR,dataHRCV-BCcn3-dataWCV-BCcn3-welchDataBC--dataHR,gaussianDatacn3BC-fftDataBC-dataFFTCV-BC--dataCorr1d,fftDatacn3BC-dataHRcn3-dataFFTCV-BC--dataCorr1d,fftDatacn3BC-dataHRCV-BCcn3-dataFFTCV-BC--dataCorr1d,gaussianDatacn3BC-fftDataBC-dataWCV-BC--dataCorr1d,fftDatacn3BC-dataHRcn3-dataWCV-BC--dataCorr1d,fftDatacn3BC-dataHRCV-BCc

In [193]:
import matplotlib.pyplot as plt
if len(nameArray) < 40:
    plt.figure(figsize=[30,10])
    plt.boxplot(topPerFeature)
    plt.legend(nameArray)
    print("")

In [194]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.8 : subject 2, feature:dataHRCV-BCcn3-dataWCV-BCcn3-dataGCVBC--welchDataBC, kernellinear, C:2.5
Accuracy 0.8 : subject 2, feature:dataHRCV-BCcn3-dataWCV-BCcn3--welchDataBC, kernellinear, C:2.5
Accuracy 0.8 : subject 2, feature:dataHRCV-BCcn3-dataWCV-BCcn3-welchDataBC--dataWCV-BC, kernellinear, C:2.5
Accuracy 0.79 : subject 5, feature:welchDatacn3-dataGCVBC-welchDataBC--dataWCV-BC, kernellinear, C:2.5
Accuracy 0.79 : subject 5, feature:dataWCV-BCcn3-welchDatacn3-dataGCVBC--welchDataBC, kernellinear, C:2.5
Accuracy 0.79 : subject 5, feature:welchDatacn3-dataGCVBC--welchDataBC, kernellinear, C:2.5
Accuracy 0.79 : subject 5, feature:dataFFTCV-BCcn3-welchDatacn3-dataGCVBC--welchDataBC, kernellinear, C:2.5
Accuracy 0.79 : subject 2, feature:dataHRCV-BCcn3-dataWCV-BCcn3-fftDatacn3--welchDataBC, kernellinear, C:2.5
Accuracy 0.79 : subject 5, feature:welchDatacn3-dataGCVBC-welchDataBC--dataFFTCV-BC, kernellinear, C:2.5
Accuracy 0.79 : subject 2, feature:dat

In [195]:
# top for each feature

In [196]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop[:1]:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.8 : subject 2, feature:dataHRCV-BCcn3-dataWCV-BCcn3-dataGCVBC--welchDataBC, kernellinear, C:2.5

Accuracy 0.79 : subject 5, feature:welchDatacn3-dataGCVBC-welchDataBC--dataWCV-BC, kernellinear, C:2.5

Accuracy 0.78 : subject 4, feature:welchDatacn3BC-fftDatacn3BC-dataWCV-BC--welchData, kernellinear, C:2.5

Accuracy 0.78 : subject 9, feature:fftDatacn3BC-dataHRCV-BCcn3-fftDatacn3--dataCorr1d, kernelsigmoid, C:2.5

Accuracy 0.77 : subject 7, feature:dataHRCV-BCcn3-welchDataBC-welchData--fftData, kernelrbf, C:2.5

Accuracy 0.75 : subject 8, feature:gaussianDatacn3BC-fftDataBC-dataFFTCV-BC--dataCorr1d, kernellinear, C:2.5

Accuracy 0.75 : subject 3, feature:fftDatacn3BC-dataHRcn3-welchDataBC--dataCorr1d, kernelsigmoid, C:2.5

Accuracy 0.74 : subject 1, feature:dataCorr1dcn3BC-fftDatacn3-fftDataBC--dataFFTCV-BC, kernellinear, C:2.5

Accuracy 0.73 : subject 6, feature:welchDatacn3BC-dataHRcn3-fftDatacn3--welchDataBC, kernelsigmoid, C:2.5


In [197]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
# print(headers[maxInd])
# print(nrOfSubjects[maxInd])
# print(list(nameDict.keys())[combinationList[maxInd][0]])
# print(list(kernelDict.keys())[combinationList[maxInd][1]])
# print(f"C: {combinationList[maxInd][2]}")


[0.         0.67342302 0.72653837 0.70112523 0.72505917 0.72845354
 0.67868257 0.71360239 0.69424888 0.71630855]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668
0,0.712005,0.713071,0.703698,0.719073,0.709933,0.716077,0.711424,0.709836,0.701629,0.714064,0.708579,0.712615,0.717646,0.709543,0.713523,0.710428,0.704505,0.701803,0.707374,0.707967,0.692652,0.701985,0.711646,0.696068,0.715693,0.708281,0.702934,0.712088,0.713507,0.706749,0.720081,0.710349,0.715038,0.708255,0.706188,0.699356,0.714743,0.709342,0.713736,0.720568,0.71119,0.714377,0.708042,0.705492,0.70045,0.708127,0.709283,0.696517,0.703848,0.713779,0.695492,0.7129,0.713433,0.706441,0.70563,0.70842,0.697116,0.714077,0.709273,0.712214,0.714042,0.708945,0.702215,0.704819,0.70704,0.695982,0.713673,0.708091,0.710887,0.718715,0.710891,0.715206,0.719461,0.708455,0.713809,0.71234,0.713586,0.703878,0.719572,0.709263,0.715979,0.710358,0.713653,0.702417,0.719073,0.70968,0.716309,0.70147,0.709216,0.699489,0.7193,0.709604,0.716834,0.726639,0.715652,0.709934,0.710659,0.702308,0.701134,0.7274,0.718556,0.714457,0.695732,0.702673,0.685487,0.708067,0.715498,0.698569,0.698719,0.709287,0.693385,0.712807,0.713617,0.70377,0.707306,0.714008,0.698102,0.700798,0.71007,0.700464,0.706925,0.710133,0.70365,0.712026,0.708475,0.702139,0.697685,0.705251,0.686505,0.705532,0.712149,0.696125,0.704125,0.709463,0.695133,0.71092,0.711445,0.703625,0.708078,0.711526,0.701425,0.722586,0.723871,0.716118,0.709808,0.715758,0.699189,0.670257,0.688634,0.668945,0.70979,0.716775,0.698538,0.701529,0.705781,0.689066,0.696128,0.7011,0.686425,0.726668,0.71742,0.71

Max average accuracy 0.7302946127946127


In [198]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboAverage.argsort()[0][-5:][::-1]]
top5Vals = comboAverage[0][comboAverage.argsort()[0][-5:][::-1]]
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

fftDatacn3BC-dataHRcn3-fftDataBC--dataCorr1d
linear
0.7302946127946127

dataHRCV-BCcn3-dataWCV-BCcn3-welchDataBC--fftDataBC
linear
0.7280120650953985

welchDatacn3BC-fftDatacn3BC-fftDataBC--welchData
linear
0.7274003928170596

welchDatacn3BC-fftDatacn3BC-welchDataBC--welchData
linear
0.7266680695847363

dataHRCV-BCcn3-welchDataBC-fftDataBC--welchData
linear
0.7266386083052752



In [199]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboTotal.argsort()[0][-5:][::-1]]
top5Vals = comboTotal[0][comboTotal.argsort()[0][-5:][::-1]] /9
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

fftDatacn3BC-dataHRcn3-fftDataBC--dataCorr1d
linear
0.7302946127946127

dataHRCV-BCcn3-dataWCV-BCcn3-welchDataBC--fftDataBC
linear
0.7280120650953985

welchDatacn3BC-fftDatacn3BC-fftDataBC--welchData
linear
0.7274003928170596

welchDatacn3BC-fftDatacn3BC-welchDataBC--welchData
linear
0.7266680695847363

dataHRCV-BCcn3-welchDataBC-fftDataBC--welchData
linear
0.7266386083052752



In [200]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668
0,6.408043,6.417639,6.333283,6.471654,6.389394,6.444697,6.402816,6.388523,6.314659,6.426578,6.37721,6.413535,6.458813,6.385884,6.421705,6.393851,6.340543,6.316225,6.366364,6.371705,6.233864,6.317866,6.404811,6.264609,6.441237,6.374533,6.326402,6.408788,6.421566,6.360745,6.480732,6.393144,6.435341,6.374293,6.355694,6.294205,6.432689,6.384078,6.423624,6.485114,6.400707,6.429394,6.372374,6.349432,6.304053,6.373144,6.383548,6.268649,6.334634,6.424015,6.259432,6.416098,6.420896,6.357967,6.350669,6.375783,6.27404,6.426692,6.38346,6.409924,6.426376,6.380505,6.319937,6.343371,6.363359,6.263838,6.423056,6.372816,6.39798,6.468434,6.398018,6.436856,6.475152,6.376098,6.42428,6.411061,6.422273,6.334899,6.476149,6.383371,6.443813,6.39322,6.422879,6.321755,6.471654,6.387121,6.44678,6.313232,6.382942,6.295404,6.473699,6.386439,6.451503,6.539747,6.440871,6.389407,6.395934,6.32077,6.310202,6.546604,6.467008,6.430114,6.261591,6.324053,6.169381,6.372601,6.439482,6.287121,6.288472,6.383586,6.240467,6.415265,6.422551,6.333927,6.365758,6.426073,6.282917,6.307184,6.390631,6.304179,6.362323,6.391199,6.332854,6.408232,6.376275,6.319255,6.279167,6.34726,6.178548,6.349785,6.409343,6.265126,6.337121,6.385164,6.256199,6.398283,6.403005,6.332626,6.372702,6.403737,6.312828,6.50327,6.514836,6.445063,6.38827,6.441818,6.292702,6.032311,6.197702,6.020505,6.388106,6.450972,6.286843,6.313763,6.352033,6.201591,6.265152,6.309899,6.177828,6.540013,6.

285
9.0
Max total significant accuracy then averaged 0.7302946127946127 
fftDatacn3BC-dataHRcn3-fftDataBC--dataCorr1d
linear
C: 2.5 


In [201]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
